In [5]:
from create_book_corefs import *

In [130]:
import sys
sys.path.append("../training/data/")

In [127]:
!python create_book_corefs.py

APassageToIndia
original count: 84 ; enhanced count: 192
Multiple match count: 3/3777
No match count: 3712/3777
Unique match count: 62/3777
original count: 84 ; enhanced count: 192
APassageToIndia
Added 1847/1847 from exp
Added 5559/7211 from booknlp
HardTimes
original count: 73 ; enhanced count: 156
Multiple match count: 9/3062
No match count: 3021/3062
Unique match count: 32/3062
original count: 73 ; enhanced count: 156
HardTimes
Added 2852/2852 from exp
Added 1838/2635 from booknlp
MansfieldPark
original count: 59 ; enhanced count: 122
Multiple match count: 11/3414
No match count: 3375/3414
Unique match count: 28/3414
original count: 59 ; enhanced count: 122
MansfieldPark
Added 3997/3997 from exp
Added 6282/7936 from booknlp
OliverTwist
original count: 185 ; enhanced count: 378
Multiple match count: 2/4395
No match count: 4234/4395
Unique match count: 159/4395
original count: 185 ; enhanced count: 378
OliverTwist
Added 5060/5060 from exp
Added 7960/12616 from booknlp
WhereAngelsFear

In [2]:
!ls /data/datasets/project-dialogism-novel-corpus/data/APassageToIndia/

APassageToIndia.entities	charInfo.dict.pkl   quotation_info.csv
APassageToIndia.name_cloze.txt	character_info.csv  quote_info.csv
APassageToIndia.named_entities	context_quotes.txt
APassageToIndia.tokens		novel_text.txt


In [3]:
import os 

In [109]:
import shutil

In [131]:
DATA_ROOT = 'pdnc_output/'
# ALIGNED_DATA_ROOT = '/data/datasets/project-dialogism-novel-corpus/aligned_data/'

for novel in ["APassageToIndia", "HardTimes", "MansfieldPark", "OliverTwist", "WhereAngelsFearToTread", "WinnieThePooh", "DarkCorners"] :
    source = pd.read_csv(f"pdnc_output/{novel}/merged_mentions.csv")
    source.to_csv(f"../data/test_pdnc_source/{novel}/mentions_used.csv", index=False)

In [126]:
!mkdir ../data/test_pdnc_source/
!mkdir ../data/test_pdnc_source/DarkCorners
!cp -r /data/datasets/project-dialogism-novel-corpus/aligned_unseen_data/DarkCorners/* ../data/test_pdnc_source/DarkCorners

mkdir: cannot create directory ‘../data/test_pdnc_source/’: File exists


In [20]:
import re
import pandas as pd 
def replace_single(text):
    
    text = re.sub('(?<![\r\n])(\r?\n|\n?\r)(?![\r\n])'," ", text) #replace single newlines

    return text

In [88]:
t = open("/data/datasets/project-dialogism-novel-corpus/data/APassageToIndia/novel_text.txt").read()
t = replace_single(t).lower()

In [89]:
t = replace_single(t).lower()

In [101]:
def get_chapter_boundaries(t) : 
    s = []
    first_match = re.finditer("prologue\n",t)
    for m in first_match: 
        s.append(m.span()[0])

    # conditions = [re.finditer("chapter [\d]+\n", t), re.finditer("chapter [ixvl]+\n", t), re.finditer("[\s]+[ixvl]+\n\n", t), re.finditer("part [\d]+\n", t), re.finditer(f"chapter ", t)]
    conditions  = [re.finditer("(chapter [^\n]+\n\n)", t), re.finditer("(part [\d\w \.]+[:]?\n\n)",t), re.finditer("(part [\d\w \.]+[:]?[ ]?[\w]+\n\n)|(chapter[:]? [\dixvl\.]+)[^\n]+\n", t), re.finditer("\n\n([\[ixvl\.\]]+)\n\n", t), re.finditer("(chapter[\s]+)\n", t) ]
    chapter_conditions = [re.finditer("^\"(chapter[:]? [\d]+)[^\n]+\n\n", t), re.finditer("(chapter[:]? [\w]+)[^\n]+\n\n", t)]
    part_conditions = [re.finditer("(part[:]? [\d]+)[^\n]+\n\n\n", t), re.finditer("(part[:]? [\w]+)[^\n]+\n\n\n", t)]
    
    chapter_conditions = [re.finditer("(chapter[:]? [\dixvl\.]+)\n|(chapter[:]? [\dixvl\.]+)[\:\.]+[^\n]+\n|(chapter\n[ ]+[\d]+)|(chapter\n[ ]+[\d]+)|(chapter[:]? [\dixvl\.]+)[^\n]+\n", t)]#, re.finditer("(chapter[:]? [\w]+)\n\n|(chapter[:]? [\w]+)[\:]+[^\n]+\n\n", t)]
    part_conditions = [re.finditer("(part[:]? [\dixvl\.]+)\n\n\n|(part[:]? [\dixvl\.]+)[\:\.]+[^\n]+\n\n\n", t)]#, re.finditer("(part[:]? [\w]+)[^\n]+\n\n\n", t)]
    weird_condition = [re.finditer("\n\n[\[]?([\dixvl.]+)[\]]?\n\n", t)]
    chapter_string_condition = ""
    for idx in range(1,100): 
        if idx < 99 : 
            chapter_string_condition += f"chapter {num2word.word(idx).lower()}\n|"
        else : 
            chapter_string_condition += f"chapter {num2word.word(idx).lower()}\n"
    chapter_string_condition = [re.finditer(chapter_string_condition, t)]
    conditions = chapter_conditions + part_conditions + weird_condition + chapter_string_condition
    for idx, cond in enumerate(conditions) : 
        u = list(cond)
        if len(u) > 0 : 
            for match in u : 
                s.append(match.span()[0])
                
    end_match = re.finditer("epilogue\n",t)
    for m in end_match :
        s.append(m.span()[0])
    s = sorted(s)
    ss = s[1:]
    sss = s[:-1]
    vals = [i-j for i,j in zip(ss,sss)]
    # choices = np.arange(len(s)).reshape(-1,2)

    cnt = 0
    for idx in range(1,len(vals)+1) : 
        if vals[idx-1] < 100 : 
            #boundaries.append(s[choices[idx][0]])
            s.pop(idx - cnt)
            cnt += 1
    
    boundaries = np.zeros(shape=(len(s), 2), dtype=int)
    boundaries[:,0] = s
    boundaries[:-1,1] = s[1:]
    boundaries[-1][-1] = len(t)
    return boundaries.tolist()

In [102]:
import num2word
import numpy as np

for novel in ["APassageToIndia", "HardTimes", "MansfieldPark", "OliverTwist", "WhereAngelsFearToTread", "WinnieThePooh"]:
    t = open(f"/data/datasets/project-dialogism-novel-corpus/data/{novel}/novel_text.txt").read()
    # t = replace_single(t).lower()
    print(novel, len(get_chapter_boundaries(t.lower())))

APassageToIndia 37
HardTimes 37
MansfieldPark 48
OliverTwist 53
WhereAngelsFearToTread 10
WinnieThePooh 10


In [7]:
DATA_ROOT = '/data/datasets/project-dialogism-novel-corpus/aligned_data/'

COREF_ROOT = 'pdnc_output'
SAVE_ROOT = COREF_ROOT

IGNORE = ['_unknowable', '_group', '_narr', 'Unknowable', '(Unknown)', 'Unknown']
PREFIXES = ['Mr.', 'Mrs.', 'Miss.', 'Lady', 'Sir', 'Mrs', 'Mr', 'Miss', 'Dr.', 'Dr', 'Madame', 'Madam', \
           'Mademoiselle', 'St.', 'St', 'Ms.', 'Ms', 'Count', 'Countess']
PREFIXES.extend([x.lower() for x in PREFIXES])

novels = []
for nf in os.scandir(DATA_ROOT):
    if os.path.isdir(nf) and nf.name[0] not in ['.', '_']:
        novels.append(nf.name)
novels = sorted(novels)

In [8]:
booknlp_process_novel(novels[0])

APassageToIndia


NameError: name 'SAVE_ROOT' is not defined